In [1]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb

import numpy as np
import os
import random

Using TensorFlow backend.


In [5]:
# Label Positive Tweets with 1

pTweets = []
with open('pos_tweets.txt', encoding='utf8') as f:
        line = f.readline()
        while line != '':
            pTweets.append([line, 1])
            line = f.readline()
        f.close()
print(pTweets)

[['" I cheer myself up when I\'m down by listening to my playlist called, Genius: Ballads and Cellos. I love my iPod and my taste of music."\n', 1], ['" just watched the movie Wanted... it was pretty darn good."\n', 1], ['" now I\'m happy "\n', 1], ['"--plotting like i\'m mike..\'game plan:pass the ball to lebron AT ALL TIMES and DONT FOUL\'..certainly we\'ll win  haha..go cavs goooo!"\n', 1], ['"@ mcdonalds with my litto sis aka cuzin lol cristyyyyy "\n', 1], ['"@ PBnJen : Thanks for the great tour and making me even more excited to work in PR!  You Rock and so does S"\n', 1], ['"@_Chelsea_Marie does target ship things to london? thanks so much! im such a demi fan shes amazing! "\n', 1], ['"@adeline_sky  that sounds fantastic! You\'re amazing! We need to watch some Muse gigs too! Shall we do it Saturday night?"\n', 1], ['"@adrianogarcia Ok, tend to use a dongle, but nice to know I could, even nicer to know iPhones can\'t  "\n', 1], ['"@agallerylondon Hi! Great to see your tweet follow

In [3]:
len(pTweets)

617

In [6]:
#Label Negative Tweets with 0

nTweets = []
with open('neg_tweets.txt', encoding='utf8') as f:
        line = f.readline()
        while line != '':
            nTweets.append([line, 0])
            line = f.readline()
        f.close()
print(nTweets)

[['" @iggigg too busy to see me in London this evening. What is a boy to do?"\n', 0], ['" cavs lost, and I got this sinking feeling we are going to lose Lebron in 2010 also...why must my home city SUCK? Ah well, LETS GO BROWNS!"\n', 0], ['" the closest BGT tour is Cardiff or London dam it why doesnt anybody other than the Chuckle Brothers tour the Westcountry?"\n', 0], ['" Why do other pet care people try to run others out of business? Or send suspicious e-mails fishing for info?"\n', 0], ['"-gasps- dananananaykroyd touring aussie ....  GRRR WHY ALWAYS SYDNEY AND MELBZ FFS"\n', 0], ['"....If i am going warsal that means no church in the morning if i dont go church but i think it imporant to see my nan coz she go cancer "\n', 0], ['"....wow the CAVS are sleeping ! I hate these blowout games "\n', 0], ['"...no more Lebron "\n', 0], ['"\'adawiyah is having fever.  hope the other 2 are not."\n', 0], ['"[-O] I wish the frogs weren\'t becoming extinct    http://tinyurl.com/cxe8w7"\n', 0], ['

In [7]:
len(nTweets)

1387

In [8]:
tweets = pTweets + nTweets

print(tweets)

[['" I cheer myself up when I\'m down by listening to my playlist called, Genius: Ballads and Cellos. I love my iPod and my taste of music."\n', 1], ['" just watched the movie Wanted... it was pretty darn good."\n', 1], ['" now I\'m happy "\n', 1], ['"--plotting like i\'m mike..\'game plan:pass the ball to lebron AT ALL TIMES and DONT FOUL\'..certainly we\'ll win  haha..go cavs goooo!"\n', 1], ['"@ mcdonalds with my litto sis aka cuzin lol cristyyyyy "\n', 1], ['"@ PBnJen : Thanks for the great tour and making me even more excited to work in PR!  You Rock and so does S"\n', 1], ['"@_Chelsea_Marie does target ship things to london? thanks so much! im such a demi fan shes amazing! "\n', 1], ['"@adeline_sky  that sounds fantastic! You\'re amazing! We need to watch some Muse gigs too! Shall we do it Saturday night?"\n', 1], ['"@adrianogarcia Ok, tend to use a dongle, but nice to know I could, even nicer to know iPhones can\'t  "\n', 1], ['"@agallerylondon Hi! Great to see your tweet follow

In [9]:
from sklearn.utils import shuffle

tweets = shuffle(tweets)
print(tweets)

[['"@seattlemommy you rock!!  i need to get motivated to run a race... "\n', 1], ['"my internect is not working fine, how do i tweet to win #squarespace iphone "\n', 0], ['"@DavidssonAsa they\'re only shipping that shield to north america "\n', 0], ['"definitely no vacation for me...   http://plurk.com/p/mzygb"\n', 0], ['"The Best Day - Taylor Swift "\n', 1], ['"Went out for just one hour in the sun yesterday..got sever headache the whole day "\n', 0], ['"laid around too much today... now my head hurts "\n', 0], ['"Wishes his xbox gets error 74 nd then he can send his xbox to germany to get it fixed for free rather than 2 red lighting all the time "\n', 0], ['"everyone go to youtube and look up "\n', 0], ['"in biology class. my lip hurts "\n', 0], ['"President obama will be arraive to ksa riyadh tomorrow....welcome there we love you so much "\n', 1], ['"I\'m tired. I feel like crap. And the world feels all crummy. Make me happy, USB disco mouse. "\n', 0], ['"omg... "\n', 0], ['"Anyone 

In [11]:
tweets = np.array(tweets)

text = tweets[:,0]
labels = tweets[:,1]

In [12]:
text

array(['"@seattlemommy you rock!!  i need to get motivated to run a race... "\n',
       '"my internect is not working fine, how do i tweet to win #squarespace iphone "\n',
       '"@DavidssonAsa they\'re only shipping that shield to north america "\n',
       ..., '"cant wait till 3am, mcdonalds breakfast "\n',
       '"Happy birthday sydney elizabeth mitchin "\n',
       '"nooooooooooo dad smashed my taylor swift cd  too bad. he\'ll have to order it again from the US"\n'],
      dtype='<U280')

In [13]:
labels

array(['1', '0', '0', ..., '1', '1', '0'], dtype='<U280')

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(text)
print(vectorizer.vocabulary_)
print(vectorizer.idf_)
vector = vectorizer.transform(text)

{'seattlemommy': 4018, 'you': 5231, 'rock': 3868, 'need': 3218, 'to': 4694, 'get': 1926, 'motivated': 3125, 'run': 3919, 'race': 3716, 'my': 3170, 'internect': 2401, 'is': 2426, 'not': 3293, 'working': 5148, 'fine': 1764, 'how': 2273, 'do': 1398, 'tweet': 4810, 'win': 5096, 'squarespace': 4327, 'iphone': 2412, 'davidssonasa': 1274, 'they': 4635, 're': 3746, 'only': 3369, 'shipping': 4103, 'that': 4607, 'shield': 4094, 'north': 3285, 'america': 298, 'definitely': 1304, 'no': 3266, 'vacation': 4921, 'for': 1825, 'me': 2964, 'http': 2282, 'plurk': 3566, 'com': 1043, 'mzygb': 3178, 'the': 4610, 'best': 569, 'day': 1277, 'taylor': 4555, 'swift': 4506, 'went': 5052, 'out': 3411, 'just': 2543, 'one': 3365, 'hour': 2269, 'in': 2366, 'sun': 4458, 'yesterday': 5223, 'got': 2007, 'sever': 4061, 'headache': 2158, 'whole': 5075, 'laid': 2644, 'around': 380, 'too': 4717, 'much': 3150, 'today': 4697, 'now': 3304, 'head': 2157, 'hurts': 2312, 'wishes': 5112, 'his': 2224, 'xbox': 5180, 'gets': 1929, 'e

In [51]:
texts = (vector.toarray())

print(texts)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [16]:
max_features = 2000
# cut texts after this number of words (among top max_features most common words)
maxlen = 80
batch_size = 32

In [56]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(texts, labels)

In [57]:
print(x_train.shape[0], 'train sequences')
print(x_test.shape[0], 'test sequences')

1503 train sequences
501 test sequences


In [58]:
x_train = sequence.pad_sequences(x_train.tolist(), maxlen = maxlen)
x_test = sequence.pad_sequences(x_test.tolist(), maxlen = maxlen)

In [20]:
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation = 'sigmoid'))

In [21]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [22]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=15, validation_data=(x_test, y_test))

Train on 1503 samples, validate on 501 samples
Epoch 1/15
1503/1503 [==============================] - 6s 4ms/step - loss: 0.6235 - acc: 0.6973 - val_loss: 0.6358 - val_acc: 0.6747
Epoch 2/15
1503/1503 [==============================] - 4s 3ms/step - loss: 0.6181 - acc: 0.6979 - val_loss: 0.6361 - val_acc: 0.6747
Epoch 3/15
1503/1503 [==============================] - 4s 3ms/step - loss: 0.6135 - acc: 0.6979 - val_loss: 0.6315 - val_acc: 0.6747
Epoch 4/15
1503/1503 [==============================] - 5s 4ms/step - loss: 0.6153 - acc: 0.6979 - val_loss: 0.6340 - val_acc: 0.6747
Epoch 5/15
1503/1503 [==============================] - 5s 3ms/step - loss: 0.6147 - acc: 0.6979 - val_loss: 0.6451 - val_acc: 0.6747
Epoch 6/15
1503/1503 [==============================] - 6s 4ms/step - loss: 0.6134 - acc: 0.6979 - val_loss: 0.6328 - val_acc: 0.6747
Epoch 7/15
1503/1503 [==============================] - 5s 3ms/step - loss: 0.6161 - acc: 0.6979 - val_loss: 0.6355 - val_acc: 0.6747
Epoch 8/15
1503

In [28]:
predictions = model.predict_classes(x_test)

print(predictions)

[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]


In [48]:
print(y_test)

['0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '1' '0' '0' '0' '1' '0' '1' '0'
 '1' '1' '0' '0' '0' '0' '1' '0' '0' '1' '0' '0' '1' '0' '0' '0' '0' '1'
 '0' '0' '0' '1' '0' '0' '0' '1' '0' '1' '0' '0' '0' '0' '0' '1' '1' '0'
 '1' '0' '0' '1' '0' '0' '1' '0' '0' '1' '0' '0' '0' '1' '1' '1' '0' '0'
 '1' '1' '0' '0' '0' '0' '1' '0' '1' '0' '0' '0' '0' '0' '1' '0' '0' '1'
 '0' '0' '0' '0' '0' '0' '1' '1' '0' '0' '0' '0' '0' '1' '0' '0' '0' '0'
 '0' '0' '0' '0' '1' '1' '0' '1' '0' '1' '0' '0' '1' '0' '0' '0' '0' '0'
 '1' '1' '1' '0' '0' '0' '0' '1' '0' '0' '0' '0' '0' '1' '1' '0' '1' '0'
 '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '1' '0' '0' '1' '0' '0'
 '0' '0' '0' '0' '1' '1' '0' '0' '0' '0' '0' '0' '1' '0' '0' '1' '0' '0'
 '0' '0' '1' '0' '0' '0' '1' '0' '0' '1' '1' '1' '0' '0' '1' '1' '0' '0'
 '1' '0' '0' '0' '0' '1' '0' '0' '0' '0' '0' '1' '0' '0' '0' '0' '0' '0'
 '1' '1' '0' '0' '1' '1' '0' '1' '1' '1' '0' '0' '1' '0' '0' '0' '1' '1'
 '1' '0' '0' '0' '0' '1' '1' '0' '0' '0' '0' '0' '1

In [ ]:
# The model seems to predicted all 0s. I think I need to improve the vectorization. 